# Plot an example starter cell

For the swc tea talk 31/03/23

In [ ]:
# imports 
%matplotlib inline
%load_ext autoreload
%autoreload 2
import iss_preprocess as iss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation

processed_path = Path(PARAMETERS['data_root']['processed'])

In [ ]:
# select cell
mouse = 'BRAC6780.3b'
chamber = 'chamber_20'
cell = 4
roi = 7

## load data

In [ ]:
# load data
data_path = f'becalia_rabies_barseq/{mouse}/{chamber}/'
metadata = iss.io.load_metadata(data_path)
ops = iss.io.load_ops(data_path=data_path)
starter_cells = pd.read_csv(processed_path / data_path / f'starter_cells_roi{roi}.csv', index_col=0)
cell_series = starter_cells.loc[cell]

In [ ]:
# define helper functions
def mcherry_in_ref(data_path, roi):
    """helper function to load mCherry and register to reference tile
    
    this uses the output as stitch_and_register
    """
    processed_path = Path(PARAMETERS["data_root"]["processed"])
    stitched = iss.pipeline.stitch.stitch_tiles(
        data_path,
        'mCherry_1',
        suffix='max',
        roi=roi,
        ich=2,
        correct_illumination=True,
    )
    tform = np.load(
    processed_path / data_path / "reg" / f"mCherry_1_roi{roi}_tform_to_ref.npz",
    allow_pickle=True,
    )
    reg_stitched = iss.pipeline.stitch.transform_image(
            stitched, scale=tform['scale'], angle=tform['angle'], shift=tform['shift']
        )
    ref_corners =  iss.pipeline.stitch.get_tile_corners(    data_path, prefix="genes_round_1_1", roi=roi)
    padding = np.array(tform['stitched_stack_shape']) - ref_corners.max(axis=(0,1,3))
    halfpad = (padding / 2).astype(int)
    reg_stitched = reg_stitched[halfpad[0]:, halfpad[1]:, ...]
    return reg_stitched

def get_spots(roi,    barcode_dot_threshold = 0.25, omp_score_threshold = 0.1, edit_distance = 2):
    """helper function to load spots and filter them
    
    genes are actually already filtered in the pkl.
    """
    # get spots
    raw_spots = dict()
    spot_list = ['barcode_round', 'genes_round']
    for prefix in spot_list:
        raw_spots[prefix] = pd.read_pickle(
            processed_path / data_path / f"{prefix}_spots_{roi}.pkl")
    raw_spots['barcode_round'].head()

    spots = dict()
    ok_barcode = raw_spots['barcode_round'].dot_product_score > barcode_dot_threshold
    spots['barcode_round'] = raw_spots['barcode_round'][ok_barcode].copy()
    ok_genes = raw_spots['genes_round'].spot_score > omp_score_threshold
    spots['genes_round'] = raw_spots['genes_round'][ok_genes].copy()
    barcode = spots['barcode_round']
    genes = spots['genes_round']
    barcode = iss.call.call.correct_barcode_sequences(barcode, max_edit_distance=2)
    return barcode, genes


In [ ]:
print('    Loading mCherry')
reg_stitched = mcherry_in_ref(data_path, roi)

In [ ]:
print('    Loading Spots')
barcode, genes = get_spots(roi)

## Plotting

In [ ]:
# define the sequence to keep
handpicked = 'TGAGTAACTG'
# how much to shift the spots to correct registration error (based on genes spots)
shift = np.array([-5, 110])
# how big of a window to show around the cell
window = np.array([-500, 500]).astype(int)

In [ ]:
# make a 2d hist with the gene rolonies
bin_width = int(2/0.1862)  # 0.186 is pixel size
genes_ppx, bx, by = np.histogram2d(
    genes.y + shift[1],
    genes.x + shift[0],
    bins=[np.arange(reg_stitched.shape[0] + 1, step=bin_width), np.arange(reg_stitched.shape[1] + 1, step=bin_width)],
)


## Overview of the start in the section

In [ ]:
# genes and mcherry
fig = plt.figure(figsize=(15, 10), facecolor="k")
xl = [100, 20000]
yl = [100, 15000]
binned_mcherry = cv2.resize(reg_stitched, (genes_ppx.shape[1], genes_ppx.shape[0]))
rng = np.max([np.diff(xl), np.diff(yl)])
step = np.max([1, rng // 4000]).astype(int)
rgb = iss.vis.to_rgb(np.dstack([binned_mcherry, genes_ppx]), 
                    colors=[(1,0,0), (0,1,1)], vmin=[50, 0], vmax=[300, 5])
ax = fig.add_subplot(1,1,1)
ax.set_facecolor("k")
ax.imshow(rgb[slice(*by.searchsorted(yl)), slice(*bx.searchsorted(xl))], extent=(xl[0], xl[1], yl[1], yl[0]), interpolation='bicubic')
ax.set_xlim(xl)
ax.set_ylim(yl[::-1])
ax.set_xticks([])
ax.set_yticks([])
ax.plot(cell_series.X, cell_series.Y, 'o', color='white', markersize=10, mfc='None')
ax.set_title(f'{mouse} {chamber} cell {cell} roi {roi}', color='white')
fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
fig.savefig(
    processed_path / data_path / "figures" / f"starter_cell{cell}_roi{roi}_overview.png", dpi=600
)
fig.savefig(
    processed_path / data_path / "figures" / f"starter_cell{cell}_roi{roi}_overview.svg", dpi=600
)

## Window around starter with mCherry background

In [ ]:
fig = plt.figure(figsize=(5, 5))
fig.set_facecolor("k")

xl = (cell_series.X + window).astype(int)
yl = (cell_series.Y + window).astype(int)


def get_red_channel(img, xl, yl, vmin=50, vmax=500):
    rng = np.max([np.diff(xl), np.diff(yl)])
    step = np.max([1, rng // 4000]).astype(int)
    rgb = iss.vis.to_rgb(
        np.dstack([img[yl[0] : yl[1] : step, xl[0] : xl[1] : step]]),
        colors=[(1, 0, 0)],
        vmin=[vmin],
        vmax=[vmax],
    )
    return rgb


rgb = get_red_channel(reg_stitched, xl, yl)
ax = plt.subplot(1, 1, 1)
img = ax.imshow(
    rgb,
    extent=(xl[0], xl[1], yl[1], yl[0]),
)

xl = np.array(xl) - shift[0]
yl = np.array(yl) - shift[1]
ok = (genes.x > xl[0]) & (genes.x < xl[1]) & (genes.y > yl[0]) & (genes.y < yl[1])
sc = ax.scatter(genes[ok].x + shift[0], genes[ok].y + shift[1], c="w", alpha=0.2, s=1)
barcode = barcode[
    (barcode.x > xl[0])
    & (barcode.x < xl[1])
    & (barcode.y > yl[0])
    & (barcode.y < yl[1])
]

seq_count = barcode.corrected_bases.value_counts()
# will plot the most common sequences in their own colors and isolated spots in orange
ns = np.sum(seq_count > 1)
cmap = plt.get_cmap("tab20", np.sum(ns))
n = 0
seq_count.sort_values(ascending=False, inplace=True)
sorted_seq = list(seq_count.index)
sorted_seq = [handpicked] + [s for s in sorted_seq if s != handpicked]

for iseq, seq in enumerate(sorted_seq):
    sdf = barcode[barcode.corrected_bases == seq]
    if len(sdf) < 2:
        color = "orange"
        label = None
    else:
        color = cmap(n)
        n += 1
        label = seq
    ax.scatter(
        sdf.x + shift[0],
        sdf.y + shift[1],
        s=10,
        alpha=0.8,
        label=label,
        ec="None",
        color=color,
    )
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
ax.set_xticks([])
ax.set_yticks([])
ax.set_title(f"{mouse} {chamber} roi {roi} cell {cell}", color="w")
fig.savefig(
    processed_path / data_path / "figures" / f"starter_cell{cell}_roi{roi}.png", dpi=600
)
fig.savefig(
    processed_path / data_path / "figures" / f"starter_cell{cell}_roi{roi}.svg", dpi=600
)


## Full cortex with all barcodes

In [ ]:
fig = plt.figure(figsize=(25, 7))
fig.set_facecolor('k')
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('k')
xl = [3500, 16000]
yl = [500, 11000]

ax.imshow(genes_ppx[slice(*by.searchsorted(yl)), slice(*bx.searchsorted(xl))],
                    cmap='Greys_r', 
                    interpolation='bicubic',
                    extent=(xl[0], xl[1], yl[1], yl[0]))
# ax.scatter(genes.x, genes.y, s=1e-6, alpha=1, c='w')

barcode = iss.call.call.correct_barcode_sequences(pd.DataFrame(barcode), max_edit_distance=2)
seq_count = barcode.corrected_bases.value_counts()
sorted_seq = list(seq_count.index)
sorted_seq = [handpicked] + [s for s in sorted_seq if s != handpicked]
cmap = plt.get_cmap('tab20', 20)
for iseq, seq in enumerate(sorted_seq):
    sdf = barcode[barcode.corrected_bases==seq]
    ax.scatter(sdf.x + shift[0], sdf.y + shift[1], s=5, alpha=0.5, ec='None', color=cmap(iseq % 20))
ax.set_aspect('equal')

ax.set_xlim(xl[0], xl[1])
ax.set_ylim(yl[1], yl[0])
ax.set_xticks([])
ax.set_yticks([])
fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
ax.set_title(f'{mouse} {chamber} roi {roi} cell {cell}', color='w')
fig.savefig(processed_path / data_path / 'figures' / f'starter_cell{cell}_roi{roi}_all_barcodes.png', dpi=1200)
fig.savefig(processed_path / data_path / 'figures' / f'starter_cell{cell}_roi{roi}_all_barcodes.svg', dpi=1200)

## Plot only selected barcode

First just to same ROI

In [ ]:
fig = plt.figure(figsize=(25, 7))
fig.set_facecolor('k')
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('k')
xl = [3500, 16000]
yl = [500, 11000]

ax.imshow(genes_ppx[slice(*by.searchsorted(yl)), slice(*bx.searchsorted(xl))],
                    cmap='Greys_r', 
                    extent=(xl[0], xl[1], yl[1], yl[0]),
                    interpolation='bicubic')
# ax.scatter(genes.x, genes.y, s=1e-6, alpha=1, c='w')

seq_count = barcode.corrected_bases.value_counts()
sorted_seq = list(seq_count.index)
sorted_seq = [handpicked] + [s for s in sorted_seq if s != handpicked]
cmap = plt.get_cmap('tab20', 20)
for iseq, seq in enumerate(sorted_seq):
    if seq != handpicked:
        continue
    sdf = barcode[barcode.corrected_bases==seq]
    ax.scatter(sdf.x, sdf.y, s=5, marker=',', alpha=0.5, ec='None', color=cmap(iseq % 20))
ax.set_aspect('equal')

ax.set_xlim(xl[0], xl[1])
ax.set_ylim(yl[1], yl[0])
ax.set_xticks([])
ax.set_yticks([])
fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
ax.set_title(f'{mouse} {chamber} roi {roi} cell {cell}', color='w')
fig.savefig(processed_path / data_path / 'figures' / f'starter_cell{cell}_roi{roi}_own_barcodes.png', dpi=1200)
fig.savefig(processed_path / data_path / 'figures' / f'starter_cell{cell}_roi{roi}_own_barcodes.svg', dpi=1200)

Now all ROIs

In [118]:
roi_list = list(range(1, 8))
roi_list.remove(6)
if False:
    img_shape = np.array([20000, 15000])
    origin_by_roi = {
        1: [4000, 2000],
        2: [0, 1000],
        3: [0, 1000],
        4: [0, 1000],
        5: [0, 1000],
        7: [0, 0],
    }
else:
    img_shape = np.array([10000, 7000])
    origin_by_roi = {
        1: [7000, 2000],
        2: [6500, 4200],
        3: [5700, 2000],
        4: [6000, 3500],
        5: [4500, 3000],
        7: [5200, 2000],
    }

fig = plt.figure(figsize=(20, 20 * 2 / 3 * img_shape[1] / img_shape[0]))
fig.set_facecolor("k")

for iroi, plot_roi in enumerate(roi_list):
    barcode, genes = get_spots(plot_roi)
    genes_ppx, bx, by = np.histogram2d(
        genes.y + shift[1],
        genes.x + shift[0],
        bins=[
            np.arange(reg_stitched.shape[0] + 1, step=bin_width),
            np.arange(reg_stitched.shape[1] + 1, step=bin_width),
        ],
    )
    ax = fig.add_subplot(2, 3, 1 + iroi, facecolor="k")
    xl = np.array([0, img_shape[0]]) + origin_by_roi[plot_roi][0]
    yl = np.array([0, img_shape[1]]) + origin_by_roi[plot_roi][1]
    ax.imshow(
        genes_ppx[slice(*by.searchsorted(yl)), slice(*bx.searchsorted(xl))],
        cmap="Greys_r",
        extent=(xl[0], xl[1], yl[1], yl[0]),
        vmax=3,
    )
    sdf = barcode[barcode.corrected_bases == handpicked]
    ax.scatter(sdf.x, sdf.y, s=7, alpha=0.7, ec="None", color=cmap(0))
    if plot_roi == 1:
        ax.set_xlim(xl[1], xl[0])
    else:
        ax.set_xlim(xl[0], xl[1])
    ax.set_ylim(yl[1], yl[0])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.5, 0.95, f"Roi {plot_roi}", color="w", transform=ax.transAxes, ha="center")
    plt.setp(ax.spines.values(), color='w')
    if plot_roi == roi:
        ax.scatter(cell_series.X - shift[0], cell_series.Y - shift[1], s=50, fc="None", color='w')

fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)

fig.savefig(
    processed_path
    / data_path
    / "figures"
    / f"starter_cell{cell}_roi{roi}_all_rois.png",
    dpi=1200,
)
fig.savefig(
    processed_path
    / data_path
    / "figures"
    / f"starter_cell{cell}_roi{roi}_all_rois.svg",
    dpi=1200,
)
